In [2]:
%config Completer.use_jedi = False

In [134]:
from IPython.display import display, clear_output

In [89]:
import numpy as np
import pandas as pd
import re

In [79]:
with open('./data/name_dict.txt','r', encoding="ISO-8859-1") as fp:
    content = fp.readlines()       
    
content = [x.strip() for x in content]

In [118]:
print('0123456789'*9)
for line in content[362:370]:
    print(line)

012345678901234567890123456789012345678901234567890123456789012345678901234567890123456789
M  Aad                                  4                                             $
M  Aadam                                          1                                   $
F  Aadje                                1                                             $
M  Ådne                      +                 1                                      $
M  Aadu                                           12                                  $
?F Aaf                                  1                                             $
F  Aafke                                4                                             $
?  Aafke                                 1                                            $


In [121]:
def get_frequencies( content ):

    count = 0
    country_list  = []
    position_list = []
    
    for line in content:
        if count == 0 : # name
            country = " ".join(line.split())
            count += 1
            
        elif count == 1: # position 
            loc = line.find('|')
            count += 1
        else:
            country_list.append(country[2:-2])
            position_list.append(loc)
            count = 0 
    return country_list, position_list
            
            
country_list, position_list = get_frequencies( content[177:342] )

In [122]:
def get_charater(content):
    
    char_convert = {}
    for line in content[54:121]:
        name_char = line[1:-1].split('=')
        letter_id = int(name_char[0])
        letter_ch = name_char[1].strip().split(' or ')
        for n, letter in enumerate(letter_ch):
            if len(letter) > 4:
                letter_ch[n] = letter.strip().split()[0]

        for n, letter in enumerate(letter_ch):
            char_convert[letter] = letter_id
    return char_convert


dict_chr = get_charater( content[54:121] )

In [125]:
len(country_list)

55

In [135]:
def get_names( content ):
    df = pd.DataFrame( columns=[ 'gender', 'name'] + country_list )
    n = 0
    for line in  content :
        name_gender =  line[0:28].split()
        freq = list(line[30:-2])
        
        if len(name_gender) == 2:   
            df.loc[n] =  name_gender + freq
            n +=1
            clear_output()
            print( len(content) - n)
            
    return df

df = get_names( content[362:] )

764


In [138]:
def replace_chr(name):
    ch = re.findall(r"\<(.*?)\>", name )
    for c in ch:
        xc = '<'+c+'>'
        n = char_convert[xc]
        name = name.replace(xc, chr(n))
    return name


for index, row in df.iterrows():
    if '<' in row['name']:
        df.loc[index, 'name']= replace_chr(row['name'])

In [139]:
for index, row in df.iterrows():
    if '+' in row['name']:
        df.loc[index, 'name'] = row['name'].replace('+',' ')  

    if '-' in row['name']:
        df.loc[index, 'name'] = row['name'].replace('-',' ')  
 

In [143]:
df.gender[df.gender == '1F'] = 1
df.gender[df.gender == '1M'] = 0

df.gender[df.gender == '?F'] = 1
df.gender[df.gender == '?M'] = 0

df.gender[df.gender == 'F'] = 1
df.gender[df.gender == 'M'] = 0

df.gender[df.gender == '?'] = 2


In [146]:
df.to_csv('first_names.csv',index=False)